<a href="https://colab.research.google.com/github/patelmeshwa99/Sentiment-Analysis/blob/master/Sentiment_Analysis_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 7000

MAXLEN = 350
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Using TensorFlow backend.


In [ ]:
# Lets look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 2,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 2,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 2,
 4,
 1153,
 9,
 194,
 775,
 7,
 2,
 2,
 349,
 2637,
 148,
 605,
 2,
 2,
 15,
 123,
 125,
 68,
 2,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 2,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 2,
 5,
 2,
 656,
 245,
 2350,
 5,
 4,
 2,
 131,
 152,
 491,
 18,
 2,
 32,
 2,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 154,
 462,
 33,
 89,
 78,
 285,
 16,
 145,
 95]

###Preprocessing
As noticed above, reviews are of different lengths. This is an issue ecause neural network requires input of same length.To do this follow the procedure below:

- if the review is greater than 350 words then trim off the extra words
- if the review is less than 350 words add the necessary amount of 0's to make it equal to 350.

keras has a function that can do this:




In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

###Creating the Model
Now it's time to create the model. I used a word embedding layer as the first layer in the model and add a layers afterwards that feeds into a dense nodes to get our predicted sentiment. 

16 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu', strides=1),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(128, activation="relu"),
    # tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
    # tf.keras.layers.Dropout(0.25)
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          112000    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         6272      
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 138,945
Trainable params: 138,945
Non-trainable params: 0
__________________________________________________

###Training
Now it's time to compile and train the model. 

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=4, validation_split=0.2)

Epoch 1/4
625/625 [==============================] - 6s 9ms/step - loss: 0.4658 - acc: 0.7610 - val_loss: 0.3204 - val_acc: 0.8614
Epoch 2/4
625/625 [==============================] - 6s 9ms/step - loss: 0.2797 - acc: 0.8820 - val_loss: 0.2825 - val_acc: 0.8890
Epoch 3/4
625/625 [==============================] - 6s 9ms/step - loss: 0.2215 - acc: 0.9095 - val_loss: 0.2776 - val_acc: 0.8926
Epoch 4/4
625/625 [==============================] - 6s 9ms/step - loss: 0.1781 - acc: 0.9291 - val_loss: 0.2860 - val_acc: 0.8934


And we'll evaluate the model on our training data to see how well it performs.

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 3s 4ms/step - loss: 0.2910 - acc: 0.8854
[0.2910405397415161, 0.8853600025177002]


###Making Predictions
We need an encoder function to convert any review that we write into that form so the model can understand it. To do that I have loaded the encodings from the dataset and use them to encode our own data.




In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [ ]:
# # while were at it lets make a decode function

# reverse_word_index = {value: key for (key, value) in word_index.items()}

# def decode_integers(integers):
#     PAD = 0
#     text = ""
#     for num in integers:
#       if num != PAD:
#         text += reverse_word_index[num] + " "

#     return text[:-1]
  
# print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  # print(encoded_text)
  pred = np.zeros((1,350))
  pred[0] = encoded_text
  # print(pred[0])
  result = model.predict(pred) 
  print(result[0])

positive_review = "medium"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.4940274]
[0.1782237]
